Sources:

 
- https://www.cs.cornell.edu/courses/cs1114/2013sp/sections/S06_convolution.pdf
- https://www.youtube.com/watch?v=FThmD4nuwhw&t=95s

# The convolution operation
In this section I will discuss the 1D convolution operation used in the Convolutional Neural Networks. 

Convolution operates on two signals for 1D, or two images for 2D. The first signal or image is the *input*, and the other (called *kernel*) serves as a "filter" over the input image, producing an output image or signal. The convolution operation then takes two images or signals and outputs a third one. 

## 1D discrete convolution

In 1D The convolution operation of two functions, f and g, is defined as:
$$
\begin{eqnarray} 
h(x) = (f * g)(x) = \int_{-\infty}^{\infty} f(\tau) \, g(x - \tau) \, d\tau
\end{eqnarray}
$$

where: 

$f$: first signal or image: the image or signal that I want to process

$g$: the filter or kernel

$h$: the signal or image convoluted

$x$: This is the independent variable representing the time or space point at which we are evaluating the convolution. It is the output variable of the convolution operation.

$\tau$: This is a dummy variable used for integration. It represents the time or space shift within the convolution operation. As we integrate over τ, we are essentially sliding the function g(t - τ) across the function f(τ). 

Discretizing the Eq. (1) using discrete vectorial functions $h[x]$, $g[x]$, $f[x]$

$$
\begin{eqnarray} 
h[x] = (f * g)[x] = \sum_{\tau=-\infty}^{\infty} f[\tau] \cdot g[x - \tau]
\end{eqnarray}
$$

Using a discrete domain and vector arrays the convolution can be done over data.

In example: 

An input function $f[x]$ can be defined as a column vector like this:

$$
f[x] = \begin{bmatrix} f[x_0] \\ f[x_1] \\ f[x_2] \\ f[x_3] \\ f[x_4] \\ \end{bmatrix}
$$

A filter function or array can be defined as a column vector (with $\tau = 0$):

$$
g[x-0] = \begin{bmatrix} 3 \\ 2 \\ 1 \\ 0 \\ 0 \\ \end{bmatrix}
$$

The filter is defined as $[3, 2, 1]$ but to work with the same dimension as the input the empty's spaces are filled with zeros.

Now with $\tau > 0$, $g[x-\tau]$ **has a special meaning**: *the slicing of the filter from top to bottom adding zeros at the top of the colum vector.*
 
$$
g[x-1] = \begin{bmatrix} 0 \\ 3 \\ 2 \\ 1 \\ 0 \\ \end{bmatrix}
$$

With this definitions, let say, in example, the third element of the $h[x]$ summation ($\tau = 2$) could be:

$$h_{3}[x] = f[2]g[x-2] =  f[x_2] \cdot \begin{bmatrix} 0 \\ 0 \\ 3 \\ 2 \\ 1 \\ \end{bmatrix} $$

With this filter the components of $h(x)$ as a vector sum of components are: 

$$ h[x] = f[0]g[x-0] + f[1]g[x-1] + f[2]g[x-2] + f[3]g[x-3] + f[4]g[x-4]$$

$$ h[x] = f[x_0] \cdot \begin{bmatrix} 3 \\ 2 \\ 1 \\ 0 \\ 0 \\ \end{bmatrix} + f[x_1] \cdot \begin{bmatrix} 0 \\ 3 \\ 2 \\ 1 \\ 0 \\ \end{bmatrix} + f[x_2] \cdot \begin{bmatrix} 0 \\ 0 \\ 3 \\ 2 \\ 1 \\ \end{bmatrix} + f[x_3] \cdot \begin{bmatrix} 0 \\ 0 \\ 0 \\ 3 \\ 2 \\ \end{bmatrix} + f[x_4] \cdot \begin{bmatrix} 0 \\ 0 \\ 0 \\ 0 \\ 3 \\ \end{bmatrix} $$

$$
h[x] = \begin{bmatrix}
3 & 0 & 0 & 0 & 0 \\
2 & 3 & 0 & 0 & 0 \\
1 & 2 & 3 & 0 & 0 \\
0 & 1 & 2 & 3 & 0 \\
0 & 0 & 1 & 2 & 3 \\
\end{bmatrix}
\cdot \begin{bmatrix} f[x_0] \\ f[x_1] \\ f[x_2] \\ f[x_3] \\ f[x_4] \\ \end{bmatrix} 
$$

### Example 1

Make the discrete convolution of $f[x] = x^2$ with $x \in \mathbb{Z} \mid 0 \leq x < 5$, applying the filter $g[x, \tau] = \begin{bmatrix} 3 \\ 2 \\ 1 \\ 0 \\ 0 \\ \end{bmatrix}$ using the definition of convolution as a dot product.

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

#### Defining the vector `f_x` $f[x]$ 

In [5]:
x = np.arange(0,5)
f_x = x**2
print(f_x)

[ 0  1  4  9 16]


#### Defining the `filter` $g[x, \tau]$ and a function that helps to construct the *sliding filter matrix* `g_matrix`

In [6]:
filter = np.array([3,2,1,0,0])
def sliding_filter_matrix(f, g):
    
    while f.shape[0] != g.shape[0]:
        g = np.insert(g, g.shape[0], 0)
        
    m = [g]
    for i in range(1,len(g)):
        g = np.insert(g, 0, 0)
        g = np.delete(g, -1)
        m.append(g)
    return np.array(m).transpose()

g_matrix = sliding_filter_matrix(f_x, filter)
print(g_matrix)

[[3 0 0 0 0]
 [2 3 0 0 0]
 [1 2 3 0 0]
 [0 1 2 3 0]
 [0 0 1 2 3]]


#### Result of the convolution as the dot product between the `g_matrix` and `f_vector`

In [7]:
convolution = np.dot(g_matrix, f_x.reshape(-1,1))
print(convolution)

[[ 0]
 [ 3]
 [14]
 [36]
 [70]]


#### Comparing the solution with Numpy.convolve()
This function is often used to convolve signals but its algorithm does not part of the definition of the convolution as a dot product between a sliced matrix and a vector f. np.convolve() works with a faster algorithm using points of overlap and has 3 modes.  
With the option 'full' the solution is similar but with this mode the response has a greater shape because the function returns the convolution of each possible point of overlap with a total shape of: (N+M-1).

In [8]:
a = f_x
b = np.array([3, 2, 1])
cn = np.convolve(a,b, mode='full')
print(cn)

[ 0  3 14 36 70 41 16]


The same numeric result can be reached if the g_matrix is completed for all sliding positions of the filter

In [9]:
full_matrix = g_matrix.copy()
new_rows = np.array([[0, 0, 0, 1, 2],[0, 0, 0, 0, 1]])
full_matrix = np.insert(full_matrix, full_matrix.shape[0], new_rows, axis=0)
print(full_matrix)

[[3 0 0 0 0]
 [2 3 0 0 0]
 [1 2 3 0 0]
 [0 1 2 3 0]
 [0 0 1 2 3]
 [0 0 0 1 2]
 [0 0 0 0 1]]


In [10]:
full_convolution = np.dot(full_matrix, f_x)
print(full_convolution)

[ 0  3 14 36 70 41 16]


With the option "same" the shape of the convolution is the maximum between the input and the filter. Note that if the filter has lesser elements that the input, the response of "same" is delayed adding zeroes to the left or the right of the filter.

In [11]:
a = f_x
b = np.array([ 3, 2, 1])
cn = np.convolve(a,b, mode='same')
print(cn)

[ 3 14 36 70 41]


In [12]:
# delay to the left
a = f_x
b = np.array([0, 3, 2, 1])
cn = np.convolve(a,b, mode='same')
print(cn)

[ 0  3 14 36 70]


### Scipy version of convolution Scipy.convolve()
For 1 dimension convolution the fastest algorithm is the Fast Fourier Transform method used in scipy. This is a good option for the convolution of large signals.

In [13]:
sig = f_x
fil = np.array([0, 3, 2, 1])
filtered = signal.convolve(sig, fil, mode='same', method="fft")
print(filtered)

[ 0  3 14 36 70]


### Convolution operation component by component
Another point of view more related on how `Numpy.convolve` work is calculating each component of the $h[x]$ solution vector. Continuing in the Example 1, for 5 convolutions in the domain of  $x \in \mathbb{Z} \mid 0 \leq x < 5$

The first component of the solution vector `h[x]` using the Eq. (2) is:

$$
h[0]  = (f * g)[0] = \sum_{\tau=0}^{4} f[\tau] \cdot g[0 - \tau]
$$

remembering that:
$$
f[x] = \begin{bmatrix} 0 \\ 1 \\ 4 \\ 9 \\ 16 \end{bmatrix},
\quad
g[x-0] = \begin{bmatrix} 3 \\ 2 \\ 1 \\ 0 \\ 0 \end{bmatrix},
\quad
g[x-1] = \begin{bmatrix} 0 \\ 3 \\ 2 \\ 1 \\ 0 \end{bmatrix},
\quad
g[x-2] = \begin{bmatrix} 0 \\ 0 \\ 3 \\ 2 \\ 1 \end{bmatrix},
\quad
g[x-3] = \begin{bmatrix} 0 \\ 0 \\ 0 \\ 3 \\ 2 \end{bmatrix},
\quad
g[x-4] = \begin{bmatrix} 0 \\ 0 \\ 0 \\ 0 \\ 3 \end{bmatrix}
$$

now in example $g[0 - 0]$ means the component 0 of the $g[x-0]$ vector

the first term of the summation for h[0]
 
$$
h_{1}[0]  = f[0] \cdot g[0 - 0] = 0 \cdot 3 = 0
$$

all the other terms of the sumation for h[0]

$$
\begin{eqnarray}
h_{2}[0]  = f[1] \cdot g[1 - 0] = 1 \cdot 0 = 0 \\
h_{3}[0]  = f[2] \cdot g[2 - 0] = 4 \cdot 0 = 0 \\
h_{4}[0]  = f[3] \cdot g[3 - 0] = 9 \cdot 0 = 0 \\
h_{5}[0]  = f[4] \cdot g[4 - 0] = 16 \cdot 0 = 0 
\end{eqnarray}
$$

With this, 

$$h[0] = 0 + 0 + 0 + 0 + 0 = 0$$

for $h[1]$:

$$
h[1]  = (f * g)[1] = \sum_{\tau=0}^{4} f[\tau] \cdot g[1 - \tau]
$$

$$
\begin{eqnarray}
h_{1}[1]  = f[0] \cdot g[0 - 1] = 0 \cdot 2 = 0 \\
h_{2}[1]  = f[1] \cdot g[1 - 1] = 1 \cdot 3 = 3 \\
h_{3}[1]  = f[2] \cdot g[2 - 1] = 4 \cdot 0 = 0 \\
h_{4}[1]  = f[3] \cdot g[3 - 1] = 9 \cdot 0 = 0 \\
h_{5}[1]  = f[4] \cdot g[4 - 1] = 16 \cdot 0 = 0 
\end{eqnarray}
$$

$$h[1] = 0 + 3 + 0 + 0 + 0 = 3$$

for $h[2]$:

$$
h[2]  = (f * g)[2] = \sum_{\tau=0}^{4} f[\tau] \cdot g[2 - \tau]
$$

$$
\begin{eqnarray}
h_{1}[2]  = f[0] \cdot g[0 - 2] = 0 \cdot 1 = 0 \\
h_{2}[2]  = f[1] \cdot g[1 - 2] = 1 \cdot 2 = 2 \\
h_{3}[2]  = f[2] \cdot g[2 - 2] = 4 \cdot 3 = 12 \\
h_{4}[2]  = f[3] \cdot g[3 - 2] = 9 \cdot 0 = 0 \\
h_{5}[2]  = f[4] \cdot g[4 - 2] = 16 \cdot 0 = 0 
\end{eqnarray}
$$

$$h[2] = 0 + 2 + 12 + 0 + 0 = 14$$

for $h[3]$:

$$
h[3]  = (f * g)[3] = \sum_{\tau=0}^{4} f[\tau] \cdot g[3 - \tau]
$$

$$
\begin{eqnarray}
h_{1}[3]  = f[0] \cdot g[0 - 3] = 0 \cdot 0 = 0 \\
h_{2}[3]  = f[1] \cdot g[1 - 3] = 1 \cdot 1 = 1 \\
h_{3}[3]  = f[2] \cdot g[2 - 3] = 4 \cdot 2 = 8 \\
h_{4}[3]  = f[3] \cdot g[3 - 3] = 9 \cdot 3 = 27 \\
h_{5}[3]  = f[4] \cdot g[4 - 3] = 16 \cdot 0 = 0 
\end{eqnarray}
$$

$$h[3] = 0 + 1 + 8 + 27 + 0 = 36$$

for $h[4]$:

$$
h[4]  = (f * g)[4] = \sum_{\tau=0}^{4} f[\tau] \cdot g[4 - \tau]
$$

$$
\begin{eqnarray}
h_{1}[4]  = f[0] \cdot g[0 - 4] = 0 \cdot 0 = 0 \\
h_{2}[4]  = f[1] \cdot g[1 - 4] = 1 \cdot 0 = 0 \\
h_{3}[4]  = f[2] \cdot g[2 - 4] = 4 \cdot 1 = 4 \\
h_{4}[4]  = f[3] \cdot g[3 - 4] = 9 \cdot 2 = 18 \\
h_{5}[4]  = f[4] \cdot g[4 - 4] = 16 \cdot 3 = 48 
\end{eqnarray}
$$

$$h[4] = 0 + 0 + 4 + 18 + 48 = 70$$

finally

$$ h[x] = \begin{bmatrix} 0 \\ 3 \\ 14 \\ 36 \\ 70 \end{bmatrix} $$

As can be seeing in the element by element operations, with this approach, the convolution can be made **reversing or flipping** the kernel vector [3, 2, 1], to [1, 2, 3] and slicing it to being operated with its corresponding region of  the $f[x]$ vector. 

For each component of the $h[x]$ result vector the operation now consist in multipy element by element the flipped kernel and its correspondent region of the input signal $f[x]$ and sum the results. 

But now, for a complete convolution the input signal must be resized and filled with zeros. To achieve this, Like `Numpy.convolve`, the size of the input signal now must be $N+M-1$. And this size will be the same for the convoluted vector $h[x]$.

$$
\begin{array}{|c|c|c|c|c|c|c|c|c|}
\hline
0 & 0 & 0 & 1 & 4 & 9 & 16 & 0 & 0 \\
\hline
1 & 2 & 3 &   &   &   &    &   &   \\
\hline
  & 1 & 2 & 3 &   &   &    &   &   \\
\hline
  &   & 1 & 2 & 3 &   &    &   &   \\
\hline
  &   &   & 1 & 2 & 3 &    &   &   \\
\hline
  &   &   &   & 1 & 2 & 3  &   &   \\
\hline
  &   &   &   &   & 1 & 2  & 3 &   \\
\hline
  &   &   &   &   &   & 1  & 2 & 3 \\
\hline
\end{array}
$$

### Code to perform a 1D convolution component by component

In [14]:
# convolution function using the component approach
def one_d_convolution(signal_1d, kernel_1d):
    
    """
    Performs a one-dimensional full convolution on a signal with respect to a kernel.
    :param signal_1d: np.Array, the input signal
    :param kernel_1d: np.Array, the kernel
    :return: np.Array, the convoluted signal
    """
    
    # Get the dimensions of the signal and kernel
    signal_length = len(signal_1d)
    kernel_length = len(kernel_1d)
    
    # Calculate the length of the output signal
    output_length = signal_length + kernel_length - 1 
    
    # Initialize the output signal
    output_1d = np.zeros(output_length)
    
    # Pad the original signal with zeros to match the full convolution output size
    padded_signal = np.pad(signal_1d, (kernel_length - 1, kernel_length - 1), mode='constant')
    
    # Flip the kernel
    flipped_kernel_1d = np.flip(kernel_1d)
    
    # Perform the full convolution operation element by element
    for i in range(output_length):
        # Extract the region of interest from the padded signal
        region = padded_signal[i:i+kernel_length]
        # Perform element-wise multiplication and sum the result
        output_1d[i] = np.sum(region * flipped_kernel_1d)
    
    return output_1d

print(one_d_convolution(f_x, np.array([3, 2, 1])))

[ 0.  3. 14. 36. 70. 41. 16.]
